In [45]:
import pandas as pd

movies=pd.read_csv(r'E:\Python\Movie Recomendation System\ml-25m\ml-25m\movies.csv')
ratings=pd.read_csv(r'E:\Python\Movie Recomendation System\ml-25m\ml-25m\ratings.csv')

print(movies.head())
print(ratings.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510


In [46]:
print(ratings.shape)
print(ratings.head())


(25000095, 4)
   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510


In [47]:
unique_users = ratings['userId'].nunique()
print(f"Number of unique users in the dataset: {unique_users}")


Number of unique users in the dataset: 162541


In [48]:
# Count of ratings per user
ratings_per_user = ratings['userId'].value_counts()

# Displaying the number of unique users and the distribution
print(f"Number of unique users in the dataset: {unique_users}")
print(f"Number of ratings per user:\n{ratings_per_user.describe()}")


Number of unique users in the dataset: 162541
Number of ratings per user:
count    162541.000000
mean        153.807932
std         268.047590
min          20.000000
25%          36.000000
50%          71.000000
75%         162.000000
max       32202.000000
Name: count, dtype: float64


In [49]:
# Count of ratings per movie
ratings_per_movie = ratings['movieId'].value_counts()

# Displaying the distribution of ratings per movie
print(f"Number of ratings per movie:\n{ratings_per_movie.describe()}")


Number of ratings per movie:
count    59047.000000
mean       423.393144
std       2477.885821
min          1.000000
25%          2.000000
50%          6.000000
75%         36.000000
max      81491.000000
Name: count, dtype: float64


In [6]:
# Data cleaning 
import re 

def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "", title)

movies['clean title']=movies['title'].apply(clean_title)

movies['genres']=movies['genres'].apply(lambda x: x.split('|'))

movies.head()


,movieId,title,genres,clean title
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story 1995
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",Jumanji 1995
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),[Comedy],Father of the Bride Part II 1995


In [17]:
duplicate_ratings = ratings[ratings.duplicated(subset=['userId', 'movieId'], keep=False)]
print(duplicate_ratings)


Empty DataFrame
Columns: [userId, movieId, rating, timestamp]
Index: []


In [2]:
import pandas as pd
from scipy.sparse import csr_matrix

# Load the ratings data (movies are already loaded from your previous code)
movies=pd.read_csv(r'E:\Python\Movie Recomendation System\ml-25m\ml-25m\movies.csv')
ratings=pd.read_csv(r'E:\Python\Movie Recomendation System\ml-25m\ml-25m\ratings.csv')
# Create a sparse matrix from the ratings dataframe
# Rows represent users, columns represent movies, values are the ratings
user_movie_sparse = csr_matrix((ratings['rating'], (ratings['userId'], ratings['movieId'])))

# Check the shape of the sparse matrix (this will not load the entire matrix into memory)
print(user_movie_sparse.shape)  # Output will be (num_users, num_movies)


(162542, 209172)


In [3]:
from sklearn.decomposition import NMF

# Apply NMF on the sparse matrix
nmf_model = NMF(n_components=20, init='random', random_state=42)

# Fit the NMF model
W = nmf_model.fit_transform(user_movie_sparse)  # User feature matrix (num_users x 20)
H = nmf_model.components_  # Movie feature matrix (20 x num_movies)

print(W.shape)  # Shape: (num_users, 20)
print(H.shape)  # Shape: (20, num_movies)


C:\Users\Rono\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


(162542, 20)
(20, 209172)


In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to search for similar movies using the NMF movie features matrix
def search_similar_movies(movie_title, num_recommendations=5):
    movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)].iloc[0]['movieId']
    
    # Get the latent feature vector for the movie
    movie_vector = H[:, movie_id]
    
    # Compute cosine similarity with all other movies
    similarity = cosine_similarity(movie_vector.reshape(1, -1), H.T).flatten()
    
    # Get top recommendations
    indices = np.argpartition(similarity, -num_recommendations)[-num_recommendations:]
    recommended_movies = movies.iloc[indices].iloc[::-1]  # Sort by similarity score

    return recommended_movies[['movieId', 'title', 'genres']]

# Example: Search similar movies to "Toy Story"
recommendations = search_similar_movies('Toy Story')
print(recommendations)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Rono\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Rono\AppData\Local\Temp\ipykernel_12912\2997503861.py", line 21, in <module>
    recommendations = search_similar_movies('Toy Story')
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Rono\AppData\Local\Temp\ipykernel_12912\2997503861.py", line 6, in search_similar_movies
    movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)].iloc[0]['movieId']
               ^^^^^^
NameError: name 'movies' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Rono\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-package

In [6]:
# Function to recommend movies for a specific user based on their latent factors
def recommend_movies_for_user(user_id, num_recommendations=20):
    # Get the user's latent feature vector
    user_vector = W[user_id - 1]  # Adjust for 0-indexing
    
    # Compute similarity between the user and all movies
    movie_scores = np.dot(H.T, user_vector)
    
    # Sort the scores and get top recommendations for the user
    already_rated = ratings[ratings['userId'] == user_id]['movieId'].values
    recommendations = np.argsort(movie_scores)[::-1]
    
    # Filter out movies the user has already rated
    recommendations = [movie for movie in recommendations if movie not in already_rated][:num_recommendations]
    
    # Get the movie details for the recommendations
    recommended_movies = movies[movies['movieId'].isin(recommendations)]
    
    return recommended_movies[['movieId', 'title', 'genres']]

# Example: Recommend movies for user 1
user_recommendations = recommend_movies_for_user(1)
print(user_recommendations)


       movieId                                    title  \
13488    69712                My Sister's Keeper (2009)   
13489    69714  Kambakkht Ishq (Incredible Love) (2009)   
62422   209171           Women of Devil's Island (1962)   

                       genres  
13488                   Drama  
13489   Action|Comedy|Musical  
62422  Action|Adventure|Drama  


In [12]:
# Function to recommend movies for a specific user based on their latent factors
def recommend_movies_for_user(user_id, num_recommendations=5):
    # Get the user's latent feature vector
    user_vector = W[user_id - 3]  # Adjust for 0-indexing
    
    # Compute similarity between the user and all movies
    movie_scores = np.dot(H.T, user_vector)
    
    # Sort the scores and get top recommendations for the user
    recommendations = np.argsort(movie_scores)[::-1]  # Sorted indices of movies by score
    
    # Get movies the user has already rated
    already_rated = ratings[ratings['userId'] == user_id]['movieId'].values
    
    # Filter out movies the user has already rated
    filtered_recommendations = [movie for movie in recommendations if movie not in already_rated]
    
    # Take the top N recommendations
    top_recommendations = filtered_recommendations[:num_recommendations]  # Get multiple recommendations
    
    # Get the movie details for the recommendations
    recommended_movies = movies[movies['movieId'].isin(top_recommendations)]
    
    # Ensure you're returning multiple recommendations
    return recommended_movies[['movieId', 'title', 'genres']]

# Example: Recommend 5 movies for user 1
user_recommendations = recommend_movies_for_user(1, num_recommendations=5)
print(user_recommendations)


      movieId                                              title  \
4201     4306                                       Shrek (2001)   
4780     4886                              Monsters, Inc. (2001)   
4790     4896  Harry Potter and the Sorcerer's Stone (a.k.a. ...   
5241     5349                                  Spider-Man (2002)   
7742     8368    Harry Potter and the Prisoner of Azkaban (2004)   

                                                 genres  
4201  Adventure|Animation|Children|Comedy|Fantasy|Ro...  
4780        Adventure|Animation|Children|Comedy|Fantasy  
4790                         Adventure|Children|Fantasy  
5241                   Action|Adventure|Sci-Fi|Thriller  
7742                             Adventure|Fantasy|IMAX  


In [16]:
# Check how many ratings user 1 has given
user_1_ratings = ratings[ratings['userId'] == 1]
print(f"User 1 has rated {len(user_1_ratings)} movies.")


User 1 has rated 70 movies.


COMBined approach

In [ ]:
import pandas as pd
from sklearn.decomposition import NMF
import numpy as np

# Load ratings data
ratings = pd.read_csv('path_to_ratings.csv')

# Pivot the ratings data to create a user-movie matrix
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating', fill_value=0)

# Fit the NMF model (NNMF)
nmf = NMF(n_components=20, init='random', random_state=42)
W = nmf.fit_transform(user_movie_matrix)  # User latent factors matrix
H = nmf.components_  # Movie latent factors matrix

# Reconstruct the full ratings matrix
predicted_ratings = np.dot(W, H)

# Create a DataFrame for predicted ratings
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=user_movie_matrix.index, columns=user_movie_matrix.columns)

# Function to get top N recommendations using NNMF
def get_nnmf_recommendations(user_id, n=10):
    # Get user's predicted ratings for all movies
    user_predicted_ratings = predicted_ratings_df.loc[user_id]
    
    # Exclude movies the user has already rated
    rated_movies = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id] > 0].index
    recommendations = user_predicted_ratings.drop(rated_movies)
    
    # Return top N movie recommendations
    top_recommendations = recommendations.sort_values(ascending=False).head(n)
    return top_recommendations.index.tolist()

